## Busqueda con retroceso

ouverture du fichier

In [40]:
def open_file(path):
    test=[]
    
    with open(path, 'r') as f:
        lignes = f.readlines()
        for i in range (0, len(lignes), 2):
            m,n = lignes[i].split()
            taille = [int(m),int(n)]
             
            
            figures = lignes[i+1].split()
            liste = [taille]
            for i in range (0,len(figures), 2):
                coordonnees = [int(figures[i])+1,int(figures[i+1])+1]
                liste.append(coordonnees)
        
            test.append(liste)
            
     
            
        return test

In [41]:
donnees = open_file("C:/Users/combl/Documents/Espagne/Algoritmo_basico/practicas/p3/test.txt")

fonction pour determiner les points auxquels il y a un check point

In [43]:
def checkpoints(taille):
    etapes = []
    for i in range(1, 4,1):
        etapes.append( i*taille[0]*taille[1]//4)
    return etapes
        

conditions à remplir:
on ne doit pas retourner là où on était deja
il faut arriver pile au bon moment sur le checkpoint
il faut verifier au point si c'est possible d'y arriver de là où on est
il faut verifier qu'il n'y ait pas de zonnes auquels on ne peut pas accéder

In [45]:
import numpy as np

Création de la première matrice avec une bordure de 1 et un centre de 0 

In [47]:
def new_matrice(taille):
    matrice = np.empty((taille[0]+2,taille[1]+2))
    for i in range (0, taille[0]+2, 1):
        for j in range(0,taille[1]+2,1):
            if (i ==0 or j == 0 or i== taille[0]+1 or j== taille[1]+1) or (i==1 and j==1):
                matrice[i][j]=1
            else :
                matrice[i][j] = 0
    return matrice
        

In [48]:
def verif_manhatan_dist (coord, point, etape, etape_point):
    dist = abs (coord[0] - int( point[0])) + abs(coord[1] - int( point[1]))
    #print("dist", dist)
    #print("nb_etapes-restantes", int(etape_point)-etape)
    if dist > (int(etape_point)-etape):
        return False
    else:
        return True
    

In [49]:
def verification_entoure(matrice):
    ##on regarde en fonction de la case 1, il doit toujours avoir la possibilité de ralier la case un en plus

    first  = True
    visite = np.zeros_like(matrice)
    voisins = [(1,0),(0,-1),(0,1),(-1,0)]
    nouveaux_voisins=[]
    
    for i,j in voisins:
        case_x = 1+i
        case_y = 1+j
        if matrice[case_x][case_y]==0:
            nouveaux_voisins.append((case_x,case_y))
            visite[case_x][case_y] = 2
    ##print("nouveaux_voisins_premiere_fois", nouveaux_voisins)
                
    if not nouveaux_voisins:
        return False

    while nouveaux_voisins:
        x,y = nouveaux_voisins.pop(0)
        for i,j in voisins:
            case_x = x+i
            case_y = y+j
            if matrice[case_x, case_y] == 0 and visite[case_x, case_y]!=2:
               visite[case_x, case_y]=2
               nouveaux_voisins.append((case_x,case_y))
        ##print("nouveaux_voisins", nouveaux_voisins)


               
    ##print(matrice)
    ##print("visite",visite)
    for i in range (1, matrice.shape[0]-1,1):
        for j in range(1, matrice.shape[1]-1):
            if matrice[i][j]==0:
                if visite[i][j]!= 2:
                    return False
                        
        
    return True
    

In [50]:
def creation_matrice_marquee(matrice, coor):
    new_matrice = np.empty((matrice.shape[0],matrice.shape[1]))
    for i in range (0,matrice.shape[0],1):
        for j in range (0,matrice.shape[1],1):
            if (i == coor[0] and j == coor[1]) :
                new_matrice[i][j]= 1
            else:
                new_matrice[i][j] = matrice[i][j]
    return new_matrice
            

In [51]:
def parcourir_voisins(liste_solution,liste_retour, etape, checkpoint, etape_actuelle):
    voisins = [(1,0),(0,-1),(0,1),(-1,0)]
    nouvelle_liste_solutions = []
    #print("solution_d'entree_dans_parcourir_voisin",liste_solution)


    #voir comment mettre à jour
    coordonnees = liste_solution[len(liste_solution)-2]
    #print("coord",coordonnees)
    matrice = liste_solution[len(liste_solution)-1]
    ##print("matrice_au_debut", matrice)


    for i in range(0, len(voisins)):
        case_x = voisins[i][0]+coordonnees[0]
        ##print("x:",case_x)
       
        case_y = voisins[i][1]+coordonnees[1]
        ##print("y:",case_y)
        if matrice[case_x][case_y]==0:
            if (case_x == int(etape[0]) and case_y == int(etape[1])):
                if etape_actuelle == checkpoint:
                    #print("je suis au checkpoint")
                    new_matrice = creation_matrice_marquee(matrice,(case_x,case_y))
                    ##print(new_matrice)
                    
                    if verification_entoure(new_matrice):
                        ##print("il n'y a pas de point bloqué")
                        ##print("avant",liste_solution)
                        liste_nouvelle_solution = liste_solution.copy()
                        liste_nouvelle_solution.pop(len(liste_solution)-1)
                        liste_nouvelle_solution.append((case_x,case_y))
                        liste_nouvelle_solution.append(new_matrice)
                        ##print("apres",liste_nouvelle_solution)
                        liste_retour.append(liste_nouvelle_solution)
                    #else:
                        ##print("je suis bloqué")
                        
                #else:
                    ##print("je suis trop tot au checkpoint")
                    
            else:
                if verif_manhatan_dist ((case_x,case_y), etape, etape_actuelle, checkpoint):
                    new_matrice = creation_matrice_marquee(matrice,(case_x,case_y))
                    ##print(new_matrice)
                    ##print("c'est encore possible pour moi")
                    
                    if verification_entoure(matrice):
                        ##print("il n'y a pas de point bloqué")
                        ##print("avant",liste_solution)
                        liste_nouvelle_solution = liste_solution.copy()
                        liste_nouvelle_solution.pop(len(liste_solution)-1)
                        liste_nouvelle_solution.append((case_x,case_y))
                        liste_nouvelle_solution.append(new_matrice)
                        ##print("apres",liste_nouvelle_solution)
                        liste_retour.append(liste_nouvelle_solution)
                   #else:
                        ##print("je suis bloqué")
                        
                #else:
                    ##print("ce n'est plus possible pour moi")
                    
           
        #else:
            ##print("la case est déjà parcourue")
    ##print("fin de voisins: nouvelle_liste",liste_retour)

    return liste_retour
        
      
              
    
    

In [52]:
def chemins_possibles(liste_solutions, etapes,  checkpoint, parcourir):
    indice=0
    ##on parcours en nombre d'étape
        ## quand on arrive au checkpoint on change d'étape à atteindre. 
    for i in range(2, parcourir+1):
        #print(checkpoint)
        #print("i et checkpoint, etape",i, checkpoint[indice],etapes[indice])
        if (i > checkpoint[indice]):
            
            indice += 1
        #print("etape_chemins_possibles", i)
        
        nouvelle_liste= []
        for solution in liste_solutions:
            #print("solution", solution)
            new_sol = parcourir_voisins(solution,nouvelle_liste, etapes[indice], checkpoint[indice], i)
            if new_sol :
                nouvelle_liste = new_sol
        #print("nouvelle liste",nouvelle_liste)
        if not nouvelle_liste:
            return []
        liste_solutions = nouvelle_liste
               

        
        #print("liste_sol_fin_de_chemin",nouvelle_liste)

    return nouvelle_liste
        
            
            
                
        
        
        
            
    
        
  


def programme_final(chemin):
    
    
    donnees = open_file(chemin)
    print("donnees:" ,donnees)

    ##creer des listes et boucler après
    taille = donnees[1][0] #meme jeux que le sujet pour l'instant
    ##print("taille:",taille)
    premier_jeu = donnees[1] #meme jeux que le sujet pour l'instant
    premier_jeu_etapes = premier_jeu[1:len(premier_jeu)]
    ##regler le porobleme du fait qu'on doit ajouter 1 partout
        
    
    checkpoint = checkpoints(taille)
    ##print("premier_jeu_checkpoint",checkpoint)
    

    matrice = new_matrice(taille)
    parcourir = taille[0]*taille[1] +1
    etapes = premier_jeu_etapes
    checkpoint.append(parcourir+1)
    etapes.append((1,1))
    
    ##initialisation 
    liste_solutions = [[(1,1),matrice]]
    ##print ("liste solutions", liste_solutions)
    ##print("parcourir:",parcourir)
    ##print("etapes:",etapes)
    ##print("checkpoint:",checkpoint)
    
    
    print(chemins_possibles(liste_solutions, etapes,  checkpoint, parcourir))
    
   
    

    
    
    

In [84]:
def programme_final(chemin, sortie):
    
    donnees = open_file(chemin)
    with open(sortie, 'w') as f:
        f.close()
        
    print("donnees:" ,donnees)
    for i in range (len(donnees)):
  
        taille = donnees[i][0]
        matrice = new_matrice(taille)
        parcourir = taille[0]*taille[1]
        
        etapes = donnees[i][1:len(donnees[i])]
        checkpoint = checkpoints(taille)
        etapes.append((1,1))
        checkpoint.append(parcourir+1)
    
    
        liste_solutions = [[(1,1),matrice]]
        if checkpoint[0]==1:
            if etapes[0]!=[1,1]:
                solution = []
            else:
                etapes.pop(0)
                checkpoint.pop(0)
                solution = chemins_possibles(liste_solutions, etapes,  checkpoint, parcourir)
        else:
            solution = chemins_possibles(liste_solutions, etapes,  checkpoint, parcourir)
        nombre_sol = len(solution)
    
        print("return",nombre_sol,solution)
        with open(sortie, 'a') as f:
            f.write(f"{nombre_sol} \n")
    
    
    

In [86]:
programme_final("C:/Users/combl/Documents/Espagne/Algoritmo_basico/practicas/p3/test.txt", "C:/Users/combl/Documents/Espagne/Algoritmo_basico/practicas/p3/resultats.txt")

donnees: [[[2, 3], [1, 1], [2, 2], [2, 3]], [[3, 6], [3, 2], [3, 5], [1, 5]], [[4, 3], [3, 1], [4, 3], [1, 3]], [[6, 6], [4, 2], [6, 5], [3, 5]]]
return 1 [[(1, 1), (2, 1), (2, 2), (2, 3), (1, 3), (1, 2), array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])]]
return 2 [[(1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (2, 4), (2, 5), (3, 5), (3, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (2, 3), (2, 2), (1, 2), array([[1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.]])], [(1, 1), (2, 1), (3, 1), (3, 2), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (3, 6), (2, 6), (1, 6), (1, 5), (2, 5), (2, 4), (1, 4), (1, 3), (1, 2), array([[1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
    